# Семантичко језгро са интеграцијом OpenBnB MCP сервера

Овај нотебук демонстрира како користити Семантичко језгро са стварним OpenBnB MCP сервером за претрагу правих Airbnb смештаја користећи MCPStdioPlugin. За приступ LLM-у користи Azure AI Foundry. Да бисте подесили променљиве окружења, можете пратити [Упутство за подешавање](/00-course-setup/README.md)


## Увоз потребних пакета


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Креирање MCP прикључка

Повезаћемо се са [OpenBnB MCP сервером](https://github.com/openbnb-org/mcp-server-airbnb) користећи MCPStdioPlugin. Овај сервер пружа функционалност претраге за Airbnb преко пакета @openbnb/mcp-server-airbnb.


## Креирање клијента

У овом примеру ћемо користити Azure AI Foundry за приступ нашем LLM-у. Уверите се да су ваши променљиве окружења правилно подешене.


## Конфигурација окружења

Подесите Azure OpenAI поставке. Уверите се да су следеће променљиве окружења постављене:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Разумевање интеграције OpenBnB MCP

Овај нотебук се повезује са **правим OpenBnB MCP сервером** који пружа стварну функционалност претраге на Airbnb-у.

### Како функционише:

1. **MCPStdioPlugin**: Користи комуникацију преко стандардног улаза/излаза са MCP сервером
2. **Прави NPM пакет**: Преузима и покреће `@openbnb/mcp-server-airbnb` преко npx-а
3. **Живи подаци**: Враћа стварне податке о некретнинама са Airbnb API-ја
4. **Откривање функција**: Агент аутоматски открива доступне функције са MCP сервера

### Доступне функције:

OpenBnB MCP сервер обично пружа:
- **search_listings** - Претрага некретнина на Airbnb-у по локацији и критеријумима
- **get_listing_details** - Добијање детаљних информација о одређеним некретнинама
- **check_availability** - Провера доступности за одређене датуме
- **get_reviews** - Преузимање рецензија за некретнине
- **get_host_info** - Добијање информација о домаћинима некретнина

### Предуслови:

- **Node.js** инсталиран на вашем систему
- **Интернет конекција** за преузимање MCP сервер пакета
- **NPX** доступан (долази уз Node.js)

### Тестирање везе:

Можете ручно тестирати MCP сервер покретањем:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Ово ће преузети и покренути OpenBnB MCP сервер, на који се Semantic Kernel затим повезује ради приступа стварним подацима са Airbnb-а.


## Покретање агента са OpenBnB MCP сервером

Сада ћемо покренути AI агента који се повезује са OpenBnB MCP сервером како би тражио стварне Airbnb смештаје у Стокхолму за 2 одрасле особе и 1 дете. Слободно измените листу `user_inputs` да бисте променили критеријуме претраге.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Резиме  
Честитамо! Успешно сте направили AI агента који се интегрише са претрагом смештаја у стварном свету користећи Model Context Protocol (MCP):  

Коришћене технологије:  
- Semantic Kernel - За креирање интелигентних агената са Azure OpenAI  
- Azure AI Foundry - За могућности LLM-а и завршетак разговора  
- MCP (Model Context Protocol) - За стандардизовану интеграцију алата  
- OpenBnB MCP Server - За реалну функционалност претраге Airbnb-а  
- Node.js/NPX - За покретање спољашњег MCP сервера  

Шта сте научили:  
- MCP интеграција: Повезивање Semantic Kernel агената са спољашњим MCP серверима  
- Приступ подацима у реалном времену: Претраживање стварних Airbnb некретнина преко живих API-ја  
- Комуникација протокола: Коришћење stdio комуникације између агента и MCP сервера  
- Откривање функција: Аутоматско откривање доступних функција са MCP сервера  
- Стримовање одговора: Хватање и бележење позива функција у реалном времену  
- HTML рендеровање: Форматирање одговора агента са стилизованим табелама и интерактивним приказима  

Следећи кораци:  
- Интеграција додатних MCP сервера (време, летови, ресторани)  
- Изградња система са више агената који комбинује MCP и A2A протоколе  
- Креирање прилагођених MCP сервера за сопствене изворе података  
- Имплементација трајног памћења разговора кроз сесије  
- Деплој агента на Azure Functions са оркестрацијом MCP сервера  
- Додавање аутентификације корисника и могућности резервације  

Кључне предности MCP архитектуре:  
- Стандардизација: Универзални протокол за повезивање AI агената са спољашњим алатима  
- Подаци у реалном времену: Приступ живим, ажурираним информацијама из различитих услуга  
- Проширивост: Лака интеграција нових извора података и алата  
- Интероперабилност: Ради на различитим AI оквирима и платформама агената  
- Раздвајање одговорности: Јасна разлика између AI логике и приступа спољашњим подацима  



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква неспоразумевања или погрешна тумачења која могу произаћи из коришћења овог превода.
